In [317]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import folium
import openrouteservice as ors
import vroom
import geopy
from geopy.distance import great_circle
import math

In [318]:
def get_token():
    load_dotenv('../.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [319]:
def get_stations():
    load_dotenv('../.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    response = requests.get(url, headers = headers).json()
    return response

In [320]:
stations = get_stations()
stations_real_time = pd.DataFrame(stations["data"])
stations_real_time[["longitude", "latitude"]] = stations_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
stations_real_time = stations_real_time.drop(["geofence", "activate", "geometry", "integrator", "reservations_count", "no_available", "tipo_estacionPBSC", "virtualDelete", "virtual_bikes", "virtual_bikes_num", "code_suburb", "geofenced_capacity", "bikesGo"], axis=1)
stations_real_time['coordinates'] = list(zip(stations_real_time['longitude'], stations_real_time['latitude']))
stations_real_time


,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
0,"Avenida del Ensanche de Vallecas, 9,",10,17,2190,2,"453 - Avenida del Ensanche de Vallecas, 9",453,27,18,-3.612530,40.370440,"(-3.61253028, 40.37043968)"
1,"Paseo de la Chopera,33,Comunidad de Madrid España",21,2,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
2,"Paseo de la Castellana nº 122,",2,21,2224,0,150 - Castellana frente a Hermanos Pinzón,150,23,05,-3.690800,40.449100,"(-3.6908, 40.4491)"
3,"Paseo de la Castellana nº 164,Comunidad de Mad...",3,20,2225,0,157- Castellana 164,157,23,05,-3.689415,40.459137,"(-3.6894151, 40.4591366)"
4,"Guetaria , 84b,",20,4,2226,1,395b - 395 - Guetaria 84b,395b,24,12,-3.715691,40.369168,"(-3.71569102, 40.36916772)"
...,...,...,...,...,...,...,...,...,...,...,...,...
610,"Calle Francisco Balseiro nº 1,",0,0,1643,3,237 - Pablo Iglesias,237,0,06,-3.710525,40.451333,"(-3.710525, 40.451333)"
611,"237- calle Francisco Balseiro,Comunidad de Mad...",0,0,2362,3,1664 - 237- calle Francisco Balseiro,1664,0,06,-3.710274,40.451682,"(-3.7102736, 40.4516819)"
612,"Calle José Abascal frente al nº 2,",6,17,1607,0,201 - Canal,201,23,07,-3.703833,40.438694,"(-3.7038333, 40.4386944)"
613,"Calle Albelda, 6,",0,0,2316,3,"528 - Albelda, 6",528,24,15,-3.657367,40.449422,"(-3.6573666, 40.4494225)"


In [321]:
district_01 = stations_real_time[stations_real_time["code_district"]== "01"]
district_02 = stations_real_time[stations_real_time["code_district"]== "02"]
district_03 = stations_real_time[stations_real_time["code_district"]== "03"]
district_04 = stations_real_time[stations_real_time["code_district"]== "04"]
district_05 = stations_real_time[stations_real_time["code_district"]== "05"]
district_06 = stations_real_time[stations_real_time["code_district"]== "06"]
district_07 = stations_real_time[stations_real_time["code_district"]== "07"]
district_08 = stations_real_time[stations_real_time["code_district"]== "08"]
district_09 = stations_real_time[stations_real_time["code_district"]== "09"]
district_10 = stations_real_time[stations_real_time["code_district"]== "10"]
district_11 = stations_real_time[stations_real_time["code_district"]== "11"]
district_12 = stations_real_time[stations_real_time["code_district"]== "12"]
district_13 = stations_real_time[stations_real_time["code_district"]== "13"]
district_14 = stations_real_time[stations_real_time["code_district"]== "14"]
district_15 = stations_real_time[stations_real_time["code_district"]== "15"]
district_16 = stations_real_time[stations_real_time["code_district"]== "16"]
district_17 = stations_real_time[stations_real_time["code_district"]== "17"]
district_18 = stations_real_time[stations_real_time["code_district"]== "18"]
district_19 = stations_real_time[stations_real_time["code_district"]== "19"]
district_20 = stations_real_time[stations_real_time["code_district"]== "20"]
district_21 = stations_real_time[stations_real_time["code_district"]== "21"]

In [322]:
district_02

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
1,"Paseo de la Chopera,33,Comunidad de Madrid España",21,2,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
35,"Paseo Virgen del Puerto, 25,",0,0,2181,3,"BiciMAD5 - Paseo Virgen del Puerto, 25",BiciMAD5,0,02,-3.721009,40.411611,"(-3.72100859, 40.41161087)"
65,"Calle Ribera de Curtidores nº 28,",9,18,1450,2,46 - Ribera de Curtidores,46,27,02,-3.707126,40.405315,"(-3.7071259, 40.4053153)"
114,Calle Teresa López Valcárcel esq. Teniente Cor...,19,5,2071,1,270 - Calle Teresa López Valcárcel,270,24,02,-3.693288,40.388629,"(-3.69328771, 40.38862908)"
157,"Calle Delicias, 45,",9,10,2117,2,"273 - Calle Delicias, 45",273,19,02,-3.690918,40.403819,"(-3.69091831, 40.40381914)"
210,"Glorieta de Embajadores nº 6,",5,19,1451,0,47 - Embajadores 1,47,24,02,-3.702800,40.404577,"(-3.7027998, 40.4045772)"
234,"PASEO DE LOS MELANCÓLICOS 73,",15,8,2247,2,280 - PASEO DE LOS MELANCÓLICOS 73,280,23,02,-3.718569,40.401782,"(-3.718569, 40.401782)"
248,"271 - Calle Amaltea , 1,",21,3,2265,1,"271 - Calle Amaltea , 1",271,24,02,-3.683170,40.394706,"(-3.68317, 40.394706)"
250,"Paseo de Juan Antonio Vallejo Nájera Botas, 28,",18,4,2267,1,"275 - Juan Antonio Vallejo Nájera Botas, 25",275,22,02,-3.708212,40.402054,"(-3.7082123, 40.4020545)"
252,"Calle Méndez Alvaro, 73,",12,11,2269,2,"274 - Méndez Alvaro, 73",274,23,02,-3.681343,40.396758,"(-3.68134346, 40.39675755)"


In [323]:
def get_light1(df):
    df_light1 = df[df["light"]==1]
    return df_light1

In [324]:
get_light1(district_02)

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
1,"Paseo de la Chopera,33,Comunidad de Madrid España",21,2,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
114,Calle Teresa López Valcárcel esq. Teniente Cor...,19,5,2071,1,270 - Calle Teresa López Valcárcel,270,24,02,-3.693288,40.388629,"(-3.69328771, 40.38862908)"
248,"271 - Calle Amaltea , 1,",21,3,2265,1,"271 - Calle Amaltea , 1",271,24,02,-3.683170,40.394706,"(-3.68317, 40.394706)"
250,"Paseo de Juan Antonio Vallejo Nájera Botas, 28,",18,4,2267,1,"275 - Juan Antonio Vallejo Nájera Botas, 25",275,22,02,-3.708212,40.402054,"(-3.7082123, 40.4020545)"
332,"Paseo virgen de puerto,",15,0,2304,1,277 - Paseo virgen de puerto,277,15,02,-3.721178,40.411622,"(-3.721178, 40.411622)"
391,"Calle del Cobre (Metro Arganzuela Planetario),",17,6,2017,1,268 - Calle del Cobre (M. Planetario),268,23,02,-3.689031,40.393383,"(-3.689031463575172, 40.39338327980599)"
409,"Calle Juan Martín el Empecinado nº 16,",20,3,1524,1,118 - Juan Martín,118,23,02,-3.688241,40.400781,"(-3.6882407, 40.400781)"
423,"Calle Sodio nº 1B,",23,0,1641,1,235 - Embajadores 191,235,23,02,-3.691076,40.389041,"(-3.691076, 40.389041)"
424,"Paseo Imperial nº 20,",25,2,1642,1,236 - Paseo Imperial,236,27,02,-3.717078,40.407579,"(-3.7170785, 40.4075794)"
528,"Paseo de los Olmos nº 28,",19,4,1568,1,162 - Metro Pirámides,162,23,02,-3.710577,40.403497,"(-3.7105765, 40.4034968)"


In [325]:
def get_light0(df):
    df_light0 = df[df["light"]==0]
    return df_light0

In [326]:
get_light0(district_02)

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
210,"Glorieta de Embajadores nº 6,",5,19,1451,0,47 - Embajadores 1,47,24,02,-3.702800,40.404577,"(-3.7027998, 40.4045772)"
411,"Calle Palos de la Frontera nº 27,",1,26,1526,0,120 - Palos de la Frontera,120,27,02,-3.694825,40.403090,"(-3.6948251018524214, 40.403090451430614)"
412,"Paseo de Santa María de la Cabeza nº 58,",7,20,1527,0,121 - Santa María de la Cabeza,121,27,02,-3.698603,40.401749,"(-3.6986027, 40.4017487)"
529,"Paseo de la Esperanza nº 2,",5,14,1569,0,163 - Paseo de la Esperanza,163,19,02,-3.706445,40.403759,"(-3.706445, 40.4037586)"
533,"Paseo Santa María de la Cabeza nº 29,",4,18,1574,0,168 - Batalla del Salado,168,22,02,-3.695900,40.404600,"(-3.6959, 40.4046)"


In [327]:
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, vehicle_start).meters)
    return nearest_station

In [328]:
distrito02_low = get_light0(district_02)
distrito02_high = get_light1(district_02)
find_nearest_to_coords(distrito02_high, current_coords)

(-3.71748917032373, 40.41374770742206)

In [330]:
load_dotenv('../.env')

client = ors.Client(key=os.environ.get("openroute_api_key"))

# Function to find the nearest station to given coordinates
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, coords).meters)
    return nearest_station

# Function to create a route between two sets of coordinates
def create_route(client, start_coords, end_coords):
    return client.directions(
        coordinates=[start_coords, end_coords],
        profile='driving-car',
        format='geojson'
    )

# Coordenadas de inicio (supongo que empiezan la ruta en la central de la EMT)
vehicle_start = [-3.6823731969472644, 40.46209827032537]

# Creo un mapa con Folium
m = folium.Map(location=vehicle_start, zoom_start=6)

# Añado marcador morado para la central (vehicle_start)
folium.Marker(location=[vehicle_start[1], vehicle_start[0]], popup='CENTRAL EMT', icon=folium.Icon(color='purple')).add_to(m)

distrito02_low = get_light0(district_02)
distrito02_high = get_light1(district_02)

# Inicio de la ruta
current_coords = vehicle_start
van = "empty"

coords_list = [current_coords]

while list(distrito02_high["light"]).count(1) > 0 or list(distrito02_low["light"]).count(0) > 0:
    if van == "empty":
        current_coords = coords_list[-1]
        distrito02_high = distrito02_high[distrito02_high['light'] != 2].reset_index(drop=True)
        if not distrito02_high.empty:
            coords_list.append(find_nearest_to_coords(distrito02_high, current_coords))
            distrito02_high.loc[distrito02_high['coordinates'] == coords_list[-1], 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "full"
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
    elif van == "full":
        current_coords = coords_list[-1]
        distrito02_low = distrito02_low[distrito02_low['light'] != 2].reset_index(drop=True)
        if not distrito02_low.empty:
            coords_list.append(find_nearest_to_coords(distrito02_low, current_coords))
            distrito02_low.loc[distrito02_low['coordinates'] == coords_list[-1], 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "empty"
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)

m

KeyboardInterrupt: 

In [366]:
load_dotenv('../.env')

client = ors.Client(key=os.environ.get("openroute_api_key"))

# Function to find the nearest station to given coordinates
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, coords).meters)
    return nearest_station

# Function to create a route between two sets of coordinates
def create_route(client, start_coords, end_coords):
    return client.directions(
        coordinates=[start_coords, end_coords],
        profile='driving-car',
        format='geojson'
    )

# Coordenadas de inicio (supongo que empiezan la ruta en la central de la EMT)
vehicle_start = [-3.6823731969472644, 40.46209827032537]

# Creo un mapa con Folium
m = folium.Map(location=[vehicle_start[1], vehicle_start[0]], zoom_start=12)

# Añado marcador morado para la central (vehicle_start)
folium.Marker(location=[vehicle_start[1], vehicle_start[0]], popup='CENTRAL EMT', icon=folium.Icon(color='purple')).add_to(m)

distrito02_low = get_light0(district_02).copy()
distrito02_high = get_light1(district_02).copy()

distrito02_high['visited'] = False
distrito02_low['visited'] = False

# Inicio de la ruta
current_coords = vehicle_start
van = "empty"

coords_list = [current_coords]

stop_counter = 1

for i in range(30): #Lo hardcodeo para que funcione
#while distrito02_high['light'].eq(1).any() or distrito02_low['light'].eq(0).any():
    if van == "empty":
        current_coords = coords_list[-1]
        # Verifica si quedan estaciones sin visitar
        if not distrito02_high.loc[~distrito02_high['visited'] & (distrito02_high['light'] == 1)].empty:
            nearest_station = find_nearest_to_coords(distrito02_high.loc[~distrito02_high['visited'] & (distrito02_high['light'] == 1)], current_coords)
            coords_list.append(nearest_station)
            # Actualiza 'visited' y 'light' en el DataFrame original
            distrito02_high.loc[distrito02_high['coordinates'] == nearest_station, 'visited'] = True
            distrito02_high.loc[distrito02_high['coordinates'] == nearest_station, 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "full"
            folium.Marker(location=[nearest_station[1], nearest_station[0]],
                          popup=f"Station with high occupation\nNumber: {stop_counter}",
                          icon=folium.Icon(color='orange')).add_to(m)
            stop_counter += 1
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
    elif van == "full":
        current_coords = coords_list[-1]
        # Verifica si quedan estaciones sin visitar
        if not distrito02_low.loc[~distrito02_low['visited'] & (distrito02_low['light'] == 0)].empty:
            nearest_station = find_nearest_to_coords(distrito02_low.loc[~distrito02_low['visited'] & (distrito02_low['light'] == 0)], current_coords)
            coords_list.append(nearest_station)
            # Actualiza 'visited' y 'light' en el DataFrame original
            distrito02_low.loc[distrito02_low['coordinates'] == nearest_station, 'visited'] = True
            distrito02_low.loc[distrito02_low['coordinates'] == nearest_station, 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "empty"
            folium.Marker(location=[nearest_station[1], nearest_station[0]],
                          popup=f"Station with low occupation\nNumber: {stop_counter}",
                          icon=folium.Icon(color='green')).add_to(m)
            stop_counter += 1
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
final_route = create_route(client, coords_list[-1], vehicle_start)
folium.PolyLine(locations=[coord[::-1] for coord in final_route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)

m